In [1]:
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## 数据，以西瓜书为例,假设下面是不同用户对一个西瓜的评价

In [2]:

path = '/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/setcase.csv'
data = np.loadtxt(path,dtype=str,delimiter=',')
data

array([['ID', '色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜'],
       ['1', '青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
       ['2', '乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好瓜'],
       ['3', '乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
       ['4', '青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好瓜'],
       ['5', '浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好瓜'],
       ['6', '青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '好瓜'],
       ['7', '乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', '好瓜'],
       ['8', '乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', '好瓜'],
       ['9', '乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', '坏瓜'],
       ['10', '青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', '坏瓜'],
       ['11', '浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', '坏瓜'],
       ['12', '浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', '坏瓜'],
       ['13', '青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', '坏瓜'],
       ['14', '浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', '坏瓜'],
       ['15', '乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '坏瓜'],
       ['16', '浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', '坏瓜'],
       ['17', '青绿', '蜷缩', '沉闷', '稍糊', '稍

In [3]:
x = data[1:,1:-1]
y = data[:,-1]

## 特征编码器 返回编码和频率

In [4]:
def features_coder(set_):
    features_set = np.unique(set_)
    frequency = [(set_==element).sum() for element in features_set]
    return {element:[code,frequency[code]] for code,element in zip(range(features_set.shape[0]),features_set)}
features_dict = features_coder(x)
features_dict

{'乌黑': [0, 6],
 '凹陷': [1, 7],
 '平坦': [2, 4],
 '模糊': [3, 3],
 '沉闷': [4, 5],
 '浅白': [5, 5],
 '浊响': [6, 10],
 '清晰': [7, 9],
 '清脆': [8, 2],
 '硬挺': [9, 2],
 '硬滑': [10, 12],
 '稍凹': [11, 6],
 '稍糊': [12, 5],
 '稍蜷': [13, 7],
 '蜷缩': [14, 8],
 '软粘': [15, 5],
 '青绿': [16, 6]}

In [5]:
datacode = np.array([[features_dict[j][0] for j in i] for i in x])
datacode

array([[16, 14,  6,  7,  1, 10],
       [ 0, 14,  4,  7,  1, 10],
       [ 0, 14,  6,  7,  1, 10],
       [16, 14,  4,  7,  1, 10],
       [ 5, 14,  6,  7,  1, 10],
       [16, 13,  6,  7, 11, 15],
       [ 0, 13,  6, 12, 11, 15],
       [ 0, 13,  6,  7, 11, 10],
       [ 0, 13,  4, 12, 11, 10],
       [16,  9,  8,  7,  2, 15],
       [ 5,  9,  8,  3,  2, 10],
       [ 5, 14,  6,  3,  2, 15],
       [16, 13,  6, 12,  1, 10],
       [ 5, 13,  4, 12,  1, 10],
       [ 0, 13,  6,  7, 11, 15],
       [ 5, 14,  6,  3,  2, 10],
       [16, 14,  4, 12, 11, 10]])

## 雅卡尔相似系数
A，B的交集元素数量/A，B并集的元素数量

In [6]:
def jaccard_similarity(setA,setB):
    return np.intersect1d(setA,setB).shape[0]/np.union1d(setA,setB).shape[0]

## 字典排序

In [7]:
def sorted_dict(dict_structure):
    return sorted(dict_structure.items(), key=lambda x: x[1], reverse=True)

### 第一行与其他行相似度排序

In [8]:
dict_ = {i:jaccard_similarity(datacode[0],datacode[i]) for i in range(datacode.shape[0])}
dict_

{0: 1.0,
 1: 0.5,
 2: 0.7142857142857143,
 3: 0.7142857142857143,
 4: 0.7142857142857143,
 5: 0.3333333333333333,
 6: 0.09090909090909091,
 7: 0.3333333333333333,
 8: 0.09090909090909091,
 9: 0.2,
 10: 0.09090909090909091,
 11: 0.2,
 12: 0.5,
 13: 0.2,
 14: 0.2,
 15: 0.3333333333333333,
 16: 0.3333333333333333}

## 加权平均法：
（不同维度的打分（例如：赞，好，坏，喜怒哀乐情绪）*余弦相似度/雅卡尔系数）

In [9]:
x = datacode
ydict = features_coder(data[:,-1])
y = np.array([ydict[i][0] for i in data[1:,-1]])

## 简单的协同过滤demo1 （雅卡尔系数）

In [10]:
def collaborative_filtering_jaccard(user,x,Lambda=1):
    scores = x.dot(np.ones_like(x[0]))*Lambda
    jaccard = np.array([jaccard_similarity(user,x[i]) for i in range(x.shape[0])])
    weighted_average = scores.dot(jaccard)/jaccard.dot(np.ones_like(jaccard))
    return weighted_average
collaborative_filtering_jaccard(x[0],x,y)

32.14502908514014

## 简单的协同过滤demo2 （皮尔森系数）

In [11]:
def cosine(vector1,vector2):
    return vector1.dot(vector2)/(np.linalg.norm(vector1) * np.linalg.norm(vector2))

In [12]:
cosine(x[0],x[0])

0.9999999999999998

In [13]:
def collaborative_filtering_cos(user,x,Lambda=1):
    scores = x.dot(np.ones_like(user))*Lambda
    cos = np.array([cosine(user,i) for i in x])
    cos = cos[cos>0]
    weighted_average = scores.dot(cos)/cos.dot(np.ones_like(cos))
    return weighted_average
collaborative_filtering_cos(x[0],x,y)

23.220019709473465

In [14]:
collaborative_filtering_cos(x[0],x)

50.10165501412123

In [15]:
tensorX = torch.Tensor(x)
tensorY = torch.Tensor(y)

In [16]:
def torch_cosine(vector1,vector2)->torch.Tensor:
    return vector1.mul(vector2).sum()/(vector1.norm()*vector2.norm())
torch_cosine(tensorX[0],tensorX[0])

tensor(1.0000)

In [17]:
def torch_collaborative_filtering_cos(user,x,Lambda=1)->torch.Tensor:
    scores = x.mm(torch.ones_like(user)[:,None]).mul(Lambda[:,None])
    cos = torch.Tensor([torch_cosine(user,i) for i in x])
    cos = cos[cos>0][:,None]
    weighted_average = scores.mul(cos).sum()/cos.sum()
    return weighted_average
torch_collaborative_filtering_cos(tensorX[0],tensorX,tensorY)

tensor(23.2200)

In [18]:
def randomMat(check,m,n,K=None)->dict:
    keys = K.keys()
    value = K.values()
    if sum([i[1] for i in value])==check:
        step1 =  [[i[0],]*i[1] for i in value]
        step2 = np.array([i for j in range(len(step1)) for i in step1[j]])
        step3 = np.random.permutation(step2)
        return step3.reshape(m,n)
    else:
        return None
newx = randomMat(6,2,3,{'a':[1,3],'b':[2,3]})

In [19]:
newx

array([[2, 2, 1],
       [2, 1, 1]])

## 自编码器

In [20]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision #数据库模块
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [21]:
test_data = torchvision.datasets.MNIST(root='./mnist/'
,train=False)
test_x = Variable(torch.unsqueeze(test_data.test_data,dim=1)
,volatile=True).type(torch.FloatTensor)[:2000]/255.
test_y = test_data.test_labels[:2000]

In [22]:
class AutoEncoder:

    def __init__(self,data):
        super(AutoEncoder,self).__init__()
        self.data = data
        self.datashape = data.shape
        self.m = self.datashape[0]
        self.h2 = self.datashape[1]
        self.m2 = self.datashape[2]
        self.n2 = self.datashape[-1]
        self.conv_layer = nn.Sequential(
        nn.Conv2d(data)
        F.relu(data)
        
        )
    

AutoEncoder(test_x).conv_layer

SyntaxError: invalid syntax (<ipython-input-22-303f0c74ee3f>, line 13)

In [23]:
test_x[0].shape

torch.Size([1, 28, 28])

In [24]:
A = [2,7,11,15]
A

[2, 7, 11, 15]

In [25]:
A = A+[A[0]]
step = {A[i]:(A[i-1],A[i]) for i in range(len(A)) if A[i]+A[i-1] ==9}
step

{7: (2, 7)}

In [26]:
for i in range(len(A)):
    if A[i]+A[i-1]==9:
        print(A[i],A[i-1])

7 2


In [27]:
step1 = [eval(z) for z in {"{}".format(sorted([i,j])) for i in A for j in A} if sum(eval(z)) == 9]
step1

[[2, 7]]